In [1]:
!pip install ta --upgrade

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=66fa24c1e065f815ebf79411662ac6bd78dc5004354cd84c2ed82c609a877cef
  Stored in directory: /root/.cache/pip/wheels/5c/a1/5f/c6b85a7d9452057be4ce68a8e45d77ba34234a6d46581777c6
Successfully built ta


In [3]:
import pandas as pd
csv_path = "/content/nifty_500.csv"
df = pd.read_csv(csv_path)
# print(df) # Keep for debugging if needed

# Extract symbols from the 'Symbol' column
# Handle potential missing values and remove leading/trailing whitespace
symbols_list = df['Symbol'].dropna().astype(str).str.strip().tolist()

# Add ".NS" to each symbol for compatibility with yfinance
symbols_list = [symbol + ".NS" for symbol in symbols_list]


# Optional: Print the list of symbols to verify
print("List of symbols extracted:")
print(symbols_list[:10]) # Print first 10 symbols as a sample
print(f"Total number of symbols: {len(symbols_list)}")

List of symbols extracted:
['3MINDIA.NS', 'AARTIDRUGS.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ADANIENT.NS']
Total number of symbols: 501


In [4]:
import math
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
# from ta.volume import OnBalanceVolume # Removed problematic import
import warnings
import os

warnings.filterwarnings("ignore")

# ---------------- CONFIG ----------------
DATA_DIR = "/content/stock_data" # Directory to save downloaded data
STARTING_CAPITAL = 25_00_000.0      # INR 25,00,000
RISK_PCT = 0.004                    # 0.4% of CURRENT total capital per entry
YEARS = 3
END_DATE = pd.Timestamp.today().normalize()
START_DATE = END_DATE - pd.DateOffset(years=YEARS)

ADX_PERIOD = 14
ATR_PERIOD = 14
DMA_FAST = 20
DMA_SLOW = 50
DMA_HIT_TOL = 0.005   # touch tolerance = 0.5% of DMA
OBV_MA_PERIOD = 20 # New: period for OBV moving average
# Ensure enough rows for all indicator calculations
MIN_ROWS = max(DMA_SLOW, ADX_PERIOD, OBV_MA_PERIOD) + 1 # Add 1 because rolling window includes current day

OUT_TRADES_CSV = "nifty_backtest_trades.csv"
OUT_SUMMARY_CSV = "nifty_backtest_summary.csv"
MAX_OPEN_POSITIONS = 250 # New: Maximum number of open positions

# ---------------- utilities ----------------
# def load_symbols_list_from_csv(csv_path):
#     df = pd.read_csv(csv_path)
#     # symbol column appears to be 'Unnamed: 3' based on your sample
#     if 'Unnamed: 3' not in df.columns:
#         raise ValueError("Expected column 'Unnamed: 3' not found in CSV.")
#     symbols_list = df['Unnamed: 3'].dropna().astype(str).str.strip().tolist()
#     # remove any stray header text "Symbol"
#     symbols_list = [s for s in symbols_list if s.upper() != "SYMBOL"]
#     # append .NS for Yahoo NSE tickers
#     yahoo = [s + ".NS" for s in symbols_list]
#     return yahoo

def fetch_and_save_price_data(ticker, start=START_DATE, end=END_DATE, data_dir=DATA_DIR):
    filepath = os.path.join(data_dir, f"{ticker}.csv")
    if os.path.exists(filepath):
        # Load existing data if available
        try:
            df = pd.read_csv(filepath, parse_dates=['Date'])
            if not df.empty and 'Date' in df.columns:
                 # Check if data is up to date
                if pd.to_datetime(df['Date'].iloc[-1]).date() >= end.date():
                    df['Date'] = pd.to_datetime(df['Date'])
                    df = df.set_index('Date')
                    return df
        except Exception as e:
            print(f"Error loading data for {ticker} from {filepath}: {e}")
            pass # redownload if loading fails

    try:
        df = yf.download(ticker, start=start.strftime("%Y-%m-%d"), end=(end + pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False, auto_adjust=False)
        if df.empty:
            print(f"No data downloaded for {ticker}")
            return None
        # ensure required columns and reset index to ensure a DatetimeIndex
        df = df[['Open','High','Low','Close','Volume']].dropna()
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)

        # Save to CSV with Date as a column
        os.makedirs(data_dir, exist_ok=True)
        df.reset_index(inplace=True)
        df.to_csv(filepath, index=False)
        df = df.set_index('Date')
        return df
    except Exception as e:
        print("Download error", ticker, e)
        return None

def load_price_data(ticker, data_dir=DATA_DIR):
    filepath = os.path.join(data_dir, f"{ticker}.csv")
    if not os.path.exists(filepath):
        return None
    try:
        # Skip the first row which contains the extra header
        df = pd.read_csv(filepath, parse_dates=['Date'], skiprows=[1])
        if not df.empty and 'Date' in df.columns:
            df = df.set_index('Date')
            # Rename columns to remove 'Ticker' level if it exists
            if isinstance(df.columns, pd.MultiIndex):
                df.columns = df.columns.droplevel(level=1)
            return df
        else:
            return None
    except Exception as e:
        print(f"Error loading data for {ticker} from {filepath}: {e}")
        return None

# Manual OBV Calculation
def calculate_obv(df):
    obv = [0] * len(df)
    for i in range(1, len(df)):
        if df['Close'].iloc[i] > df['Close'].iloc[i-1]:
            obv[i] = obv[i-1] + df['Volume'].iloc[i]
        elif df['Close'].iloc[i] < df['Close'].iloc[i-1]:
            obv[i] = obv[i-1] - df['Volume'].iloc[i]
        else:
            obv[i] = obv[i-1]
    return pd.Series(obv, index=df.index)

def compute_indicators(df):
    df = df.copy()
    # Debugging: Print index type
    print(f"Index type in compute_indicators: {type(df.index)}")
    # Ensure DataFrame has a valid index before computing indicators
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("Input DataFrame to compute_indicators must have a DatetimeIndex")

    try:
        df['dma20'] = df['Close'].rolling(window=DMA_FAST).mean()
        df['dma50'] = df['Close'].rolling(window=DMA_SLOW).mean()
        atr = AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=ATR_PERIOD)
        df['atr'] = atr.average_true_range()
        adx_indicator = ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=ADX_PERIOD)
        df['adx'] = adx_indicator.adx()
        df['+di'] = adx_indicator.adx_pos()  # Calculate +DI
        df['-di'] = adx_indicator.adx_neg()  # Calculate -DI

        # New: Compute On-Balance Volume manually and its moving average
        df['obv'] = calculate_obv(df) # Uncommented OBV calculation
        df['obv_ma20'] = df['obv'].rolling(window=OBV_MA_PERIOD).mean() # Uncommented OBV MA calculation
    except Exception as e:
        print(f"Error computing indicators: {e}")
        raise e # Re-raise the exception after printing

    return df

def touch_dma(close, dma):
    if pd.isna(dma) or pd.isna(close):
        return False
    return abs(close - dma) <= (DMA_HIT_TOL * dma)

In [5]:
# backtest_symbol function modified to only generate trade signals
def backtest_symbol(df, symbol):
    trades = []   # record per-exit trade logs
    entries = []  # open entries: list of dicts {entry_price, entry_date, entry_type}
    # We allow up to 2 entries per symbol per life-cycle (20dma entry + possible 50dma add)
    for idx in range(len(df)):
        date = df.index[idx]
        row = df.iloc[idx]
        # skip until enough data for all indicators
        if idx < MIN_ROWS:
            continue
        price = row['Close']
        adx = row['adx']
        plus_di = row['+di']  # Get +DI
        minus_di = row['-di'] # Get -DI
        dma20 = row['dma20']
        dma50 = row['dma50']
        atr = row['atr']
        obv = row['obv'] # Get OBV
        obv_ma20 = row['obv_ma20'] # Get OBV MA20


        if any(pd.isna(x) for x in [price, adx, plus_di, minus_di, dma20, dma50, atr, obv, obv_ma20]): # Add +DI, -DI, OBV and OBV MA to check
            continue

        # compute the SL level for the current day (50dma - ATR)
        sl_level = dma50 - atr

        # ---------- check stop loss first ----------
        if entries:
            # If today's close breaches SL -> exit all entries at today's close
            if price <= sl_level:
                # Record each entry's exit
                for e in entries:
                    trades.append({
                        'symbol': symbol,
                        'entry_date': e['entry_date'],
                        'entry_price': e['entry_price'],
                        'exit_date': date,
                        'exit_price': price,
                        'reason': 'SL',
                        'entry_type': e['entry_type'],
                        'atr': atr, # Include ATR in trade signal
                        'entry_adx': e['entry_adx'] # Include ADX at entry
                    })
                entries = []
                # continue to next day
                continue

            # check DMA cross: if previously dma20 > dma50 and now dma20 < dma50 => trend flipped => exit
            if idx >= 1:
                prev = df.iloc[idx-1]
                prev_dma20 = prev['dma20']; prev_dma50 = prev['dma50']
                if prev_dma20 > prev_dma50 and dma20 < dma50:
                    # exit all at today's close
                    for e in entries:
                        trades.append({
                            'symbol': symbol,
                            'entry_date': e['entry_date'],
                            'entry_price': e['entry_price'],
                            'exit_date': date,
                            'exit_price': price,
                            'reason': 'DMA_CROSS',
                            'entry_type': e['entry_type'],
                            'atr': atr, # Include ATR in trade signal
                            'entry_adx': e['entry_adx'] # Include ADX at entry
                        })
                    entries = []
                    continue


        # ---------- no open entries: consider entry at 20DMA ----------
        if not entries:
            # require ADX > 25 and upward trend (dma20 > dma50) AND price touches 20dma AND OBV > OBV_MA20 AND +DI > -DI
            if adx > 25 and dma20 > dma50 and touch_dma(price, dma20) and obv > obv_ma20 and plus_di > minus_di:
                # open first entry
                entries.append({
                    'entry_price': price,
                    'entry_date': date,
                    'entry_type': '20dma',
                    'entry_adx': adx # Record ADX at entry
                })
                continue

        # ---------- if have one entry already, allow pyramid at 50DMA ----------
        if entries and len(entries) == 1:
            # require ADX > 25 and same upward trend AND price touches 50dma AND OBV > OBV_MA20 AND +DI > -DI
            if adx > 25 and dma20 > dma50 and touch_dma(price, dma50) and obv > obv_ma20 and plus_di > minus_di:
                entries.append({
                    'entry_price': price,
                    'entry_date': date,
                    'entry_type': '50dma',
                    'entry_adx': adx # Record ADX at entry
                })
                continue

    # ------------- end loop days: if any entries remain, close at last close -------------
    if entries:
        last_price = df.iloc[-1]['Close']
        last_adx = df.iloc[-1]['adx'] # Get last ADX
        for e in entries:
            trades.append({
                'symbol': symbol,
                'entry_date': e['entry_date'],
                'entry_price': e['entry_price'],
                'exit_date': df.index[-1],
                'exit_price': last_price,
                'reason': 'END',
                'entry_type': e['entry_type'],
                'atr': atr, # Include ATR in trade signal
                'entry_adx': e['entry_adx'] # Include ADX at entry
            })
        entries = []

    # Check for duplicate entries in the 'trades' list
    seen = set()
    duplicates = []
    for trade in trades:
        # Create a tuple of the trade's key attributes to check for duplicates
        trade_key = (trade['symbol'], trade['entry_date'], trade['entry_price'], trade['exit_date'], trade['exit_price'], trade['reason'], trade['entry_type'])
        if trade_key in seen:
            duplicates.append(trade)
        else:
            seen.add(trade_key)

    if duplicates:
        print(f"Duplicate trade entries found for {symbol}:")
        for dup in duplicates:
            print(dup)

    # Return the list of trade signals
    return trades

In [6]:
# Load symbols_list from CSV
import pandas as pd
csv_path = "/content/nifty_500.csv"
df = pd.read_csv(csv_path)
# print(df) # Keep for debugging if needed

# Extract symbols from the 'Symbol' column
# Handle potential missing values and remove leading/trailing whitespace
symbols_list = df['Symbol'].dropna().astype(str).str.strip().tolist()

# Add ".NS" to each symbol for compatibility with yfinance
symbols_list = [symbol + ".NS" for symbol in symbols_list]

print("Tickers loaded:", len(symbols_list))

# First, download and save all data
print("Downloading and saving price data...")
downloaded_symbols_list = []
for sym in tqdm(symbols_list, desc="Downloading"):
    df = fetch_and_save_price_data(sym)
    if df is not None and df.shape[0] >= MIN_ROWS:
        downloaded_symbols_list.append(sym)

print(f"Successfully downloaded data for {len(downloaded_symbols_list)} symbols_list.")

# Collect all potential trade signals from backtest_symbol
all_trade_signals = []

print("Collecting trade signals using saved data...")
for sym in tqdm(downloaded_symbols_list, desc="Collecting Signals"):
    df = load_price_data(sym)
    if df is None or df.shape[0] < MIN_ROWS:
        print(f"Skipping {sym} due to insufficient data after loading.")
        continue

    try:
        df = compute_indicators(df)
        df.attrs['symbol'] = sym
        # Call backtest_symbol to get potential trade signals
        trade_signals = backtest_symbol(df, sym)

        # Append signals from this symbol
        for t in trade_signals:
            all_trade_signals.append(t)

    except ValueError as e:
        print(f"Skipping symbol {sym} due to error during indicator computation: {e}")
        continue
    except Exception as e:
        print(f"An unexpected error occurred for symbol {sym}: {e}")
        continue

all_trade_signals_df = pd.DataFrame(all_trade_signals)

# Display the DataFrame
display(all_trade_signals_df)

# Remove duplicate trade signals before simulating
print("Removing duplicate trade signals...")
seen_signals = set()
unique_trade_signals = []
for signal in all_trade_signals:
    # Create a unique key for each signal based on relevant attributes
    signal_key = (signal['symbol'], signal['entry_date'], signal['entry_price'], signal['exit_date'], signal['exit_price'], signal['reason'], signal['entry_type'])
    if signal_key not in seen_signals:
        unique_trade_signals.append(signal)
        seen_signals.add(signal_key)
    # Optional: Print detected duplicates
    # else:
    #     print(f"Detected duplicate signal: {signal}")

print(f"Original number of signals: {len(all_trade_signals)}")
print(f"Number of unique signals: {len(unique_trade_signals)}")

Tickers loaded: 501


Downloading:   2%|▏         | 12/501 [00:03<02:25,  3.37it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['ADANITRANS.NS']: YFTzMissingError('possibly delisted; no timezone found')
Downloading:   3%|▎         | 13/501 [00:04<02:45,  2.96it/s]

No data downloaded for ADANITRANS.NS


Downloading:   3%|▎         | 14/501 [00:04<02:26,  3.33it/s]ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AEGISCHEM.NS']: YFTzMissingError('possibly delisted; no timezone found')
Downloading:   3%|▎         | 16/501 [00:04<02:19,  3.47it/s]

No data downloaded for AEGISCHEM.NS


...

Index type in compute_indicators: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Index type in compute_indicators: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


Index type in compute_indicators: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Index type in compute_indicators: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


Index type in compute_indicators: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Index type in compute_indicators: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>


,symbol,entry_date,entry_price,exit_date,exit_price,reason,entry_type,atr,entry_adx
0,3MINDIA.NS,2023-07-06,27361.449219,2023-10-23,29390.050781,SL,20dma,752.326712,35.385327
1,3MINDIA.NS,2024-07-23,38490.750000,2024-08-14,36019.101562,SL,20dma,1215.446809,29.315651
2,3MINDIA.NS,2025-05-06,29640.000000,2025-06-18,29175.000000,DMA_CROSS,20dma,582.702595,28.898850
3,AARTIDRUGS.NS,2023-06-21,458.049988,2023-09-26,548.400024,SL,20dma,20.317369,37.373487
4,AARTIDRUGS.NS,2023-06-26,449.899994,2023-09-26,548.400024,SL,50dma,20.317369,30.949842
...,...,...,...,...,...,...,...,...,...
1598,ZYDUSWELL.NS,2023-09-08,320.459991,2023-10-16,312.230011,DMA_CROSS,20dma,5.845193,36.271793
1599,ZYDUSWELL.NS,2024-01-11,328.619995,2024-02-13,313.929993,SL,20dma,7.509693,31.598665
1600,ZYDUSWELL.NS,2024-06-03,344.910004,2024-09-17,433.989990,SL,20dma,12.644496,37.924001
1601,ZYDUSWELL.NS,2024-12-13,402.390015,2025-01-09,390.410004,DMA_CROSS,20dma,10.866398,26.896038


Removing duplicate trade signals...
Original number of signals: 1603
Number of unique signals: 1603


In [7]:
# Analyze unique_trade_signals for risk-reward ratio compliance

compliant_trades_count = 0
non_compliant_trades_count = 0
compliant_trades = []
non_compliant_trades = []

ATR_MULTIPLIER = 1.5
MIN_RISK_REWARD = 1.5

for signal in unique_trade_signals:
    entry_price = signal['entry_price']
    exit_price = signal['exit_price']
    atr = signal.get("atr", None)

    if atr is None or atr == 0:
        # print(f"Skipping signal for {signal['symbol']} on {signal['entry_date']} due to missing or zero ATR.")
        non_compliant_trades_count += 1 # Consider trades with no valid ATR as non-compliant
        non_compliant_trades.append(signal)
        continue

    risk_per_share = atr * ATR_MULTIPLIER
    # Use abs(exit_price - entry_price) as a proxy for potential reward based on the signal's exit price
    expected_reward_per_share = abs(exit_price - entry_price)
    risk_reward_ratio = expected_reward_per_share / risk_per_share if risk_per_share != 0 else 0

    if risk_reward_ratio >= MIN_RISK_REWARD:
        compliant_trades_count += 1
        compliant_trades.append(signal)
    else:
        non_compliant_trades_count += 1
        non_compliant_trades.append(signal)

print(f"Total unique trade signals: {len(unique_trade_signals)}")
print(f"Trades complying with Risk-Reward >= {MIN_RISK_REWARD}: {compliant_trades_count}")
print(f"Trades not complying with Risk-Reward < {MIN_RISK_REWARD} (or with no valid ATR): {non_compliant_trades_count}")

# Optional: Display compliant and non-compliant trades DataFrames
# compliant_trades_df = pd.DataFrame(compliant_trades)
# non_compliant_trades_df = pd.DataFrame(non_compliant_trades)

# print("\nCompliant Trades Sample (first 5):")
# display(compliant_trades_df.head())

# print("\nNon-Compliant Trades Sample (first 5):")
# display(non_compliant_trades_df.head())

Total unique trade signals: 1603
Trades complying with Risk-Reward >= 1.5: 634
Trades not complying with Risk-Reward < 1.5 (or with no valid ATR): 969


In [10]:
import math
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
import warnings
import os

warnings.filterwarnings("ignore")

# ---------------- CONFIG ----------------
DATA_DIR = "/content/stock_data" # Directory to save downloaded data
STARTING_CAPITAL = 25_00_000.0      # INR 25,00,000
RISK_PCT = 0.005                    # 0.5% of CURRENT total capital per entry
YEARS = 3
END_DATE = pd.Timestamp.today().normalize()
START_DATE = END_DATE - pd.DateOffset(years=YEARS)

ADX_PERIOD = 14
ATR_PERIOD = 14
DMA_FAST = 20
DMA_SLOW = 50
DMA_HIT_TOL = 0.005   # touch tolerance = 0.5% of DMA
OBV_MA_PERIOD = 20 # New: period for OBV moving average
# Ensure enough rows for all indicator calculations
MIN_ROWS = max(DMA_SLOW, ADX_PERIOD, OBV_MA_PERIOD) + 1 # Add 1 because rolling window includes current day

OUT_TRADES_CSV = "nifty_backtest_trades.csv"
OUT_SUMMARY_CSV = "nifty_backtest_summary.csv"
MAX_OPEN_POSITIONS = 250 # New: Maximum number of open positions

# New: Risk-reward constants
ATR_MULTIPLIER = 1.5
MIN_RISK_REWARD = 1.5


def simulate_trades(trade_signals, starting_capital):
    """
    Simulates trades chronologically based on trade signals, managing capital
    and open positions dynamically.

    Args:
        trade_signals: A list of dictionaries, where each dictionary represents
                       a potential trade signal with at least 'entry_date',
                       'exit_date', 'entry_price', 'exit_price', 'symbol', and 'atr'.
        starting_capital: The initial capital for the simulation.

    Returns:
        A tuple containing:
        - executed_trades: A list of dictionaries for trades that were executed.
        - final_capital: The capital at the end of the simulation.
        - final_open_positions_count: The number of open positions at the end.
        - date_wise_accounts: A list of dictionaries with date and unallocated capital.
        - largest_loss_streak_start_date: The start date of the largest loss streak.
        - largest_loss_streak_end_date: The end date of the largest loss streak.
    """
    max_positions = 0
    max_capital = starting_capital
    current_capital = starting_capital
    unallocated_capital = starting_capital
    executed_trades = []
    open_positions = [] # To store details of currently open trades {symbol, entry_date, entry_price, size, exit_date, exit_price, reason, entry_type}
    open_positions_count = 0
    date_wise_accounts = [] # New: To store date and unallocated capital



    # Sort trade signals by entry date
    trade_signals.sort(key=lambda x: x['entry_date'])

    # Get all unique dates involved in trade signals
    all_dates = sorted(list(set([t['entry_date'] for t in trade_signals] + [t['exit_date'] for t in trade_signals])))

    # Iterate through dates chronologically
    for current_date in all_dates:
        # Process exits for the current date
        exited_today_indices = [i for i, p in enumerate(open_positions) if p['exit_date'] == current_date]

        # Process exits from latest to earliest index to avoid index issues after removal
        for i in sorted(exited_today_indices, reverse=True):
            exited_pos = open_positions.pop(i) # Remove the position from open_positions
            pnl = (exited_pos['exit_price'] - exited_pos['entry_price']) * exited_pos['size']
            current_capital += pnl
            unallocated_capital += (exited_pos['entry_price'] * exited_pos['size']) + pnl
            executed_trades.append({
                'symbol': exited_pos['symbol'],
                'entry_date': exited_pos['entry_date'],
                'entry_price': exited_pos['entry_price'],
                'exit_date': exited_pos['exit_date'],
                'exit_price': exited_pos['exit_price'],
                'size': exited_pos['size'],
                'pnl': pnl,
                'reason': exited_pos['reason'],
                'entry_type': exited_pos['entry_type']
            })
            open_positions_count -= 1


        max_capital = max(max_capital, unallocated_capital)
        max_positions = max(max_positions, open_positions_count)
        # Process entries for the current date
        entries_today = [t for t in trade_signals if t['entry_date'] == current_date]

        for entry_signal in entries_today:
            # Check if we can open a new position and if there is enough unallocated capital
            # Find the corresponding data row to get price and atr for SL calculation
            # This requires access to the original price data, which is not directly
            # available in this function. For simulation purposes within this function,
            # we will use the exit_price from the trade signal as the SL level
            # if the reason is 'SL' or 'END'. This is a simplification.
            # A more robust simulation would require passing the full price data.

            # Assuming exit_price in signal is the theoretical stop or end price
            stop_price = entry_signal['exit_price'] # Simplified SL for simulation

            # Need entry price to calculate risk per share
            entry_price = entry_signal['entry_price']

            # New: Risk-reward check
            atr = entry_signal.get("atr", None)
            if atr is None or atr == 0:
                continue # Skip trades with missing or zero ATR

            risk_per_share = atr * ATR_MULTIPLIER
            expected_reward_per_share = abs(stop_price - entry_price) # Using stop_price as a proxy for expected exit price
            risk_reward_ratio = expected_reward_per_share / risk_per_share if risk_per_share != 0 else 0

            if risk_reward_ratio < MIN_RISK_REWARD:
                continue # Skip trades with low risk-reward ratio


            # # Risk per trade = 0.5% of current capital
            risk_capital = RISK_PCT * max_capital  # Use max_capital for position sizing

            # # ATR-based sizing (ATR should come from entry_signal)
            # atr = entry_signal.get("atr", None)
            # if atr is None or atr == 0: # Handle case where ATR is 0 or None
            #     # fallback: treat as flat size if ATR missing or zero
            #     size = math.floor(risk_capital / entry_price) if entry_price != 0 else 0 # Handle zero entry price
            # else:
            #     atr_multiplier = 1.5  # can tune this
            #     risk_per_share = atr * atr_multiplier
            #     size = math.floor(risk_capital / risk_per_share) if risk_per_share != 0 else 0 # Handle zero risk per share

            size = math.floor(0.02*max_capital/entry_price)
            # Calculate cost of the potential trade
            trade_cost = size * entry_price


            if unallocated_capital >= trade_cost and size > 0: # Ensure size is greater than 0
                 # Deduct trade cost from unallocated capital at entry
                unallocated_capital -= trade_cost
                open_positions.append({
                    'symbol': entry_signal['symbol'],
                    'entry_date': entry_signal['entry_date'],
                    'entry_price': entry_price,
                    'size': size,
                    'exit_date': entry_signal['exit_date'], # Store planned exit date/price
                    'exit_price': stop_price, # Store planned exit price (SL or END)
                    'reason': entry_signal['reason'],
                    'entry_type': entry_signal['entry_type']
                })
                open_positions_count += 1
                # Note: Total capital is not changed at entry, only unallocated capital

        # Calculate total portfolio value for the current date
        total_portfolio_value = unallocated_capital
        for pos in open_positions:
            # To get the current market value of open positions, we need the closing price
            # for each symbol on the current_date. This function doesn't have access
            # to the full historical dataframes. A simplified approach is to use the
            # entry price as a proxy for the current price, or if available, the
            # exit price stored in the signal (which is the price on the exit date).
            # A proper backtest simulation would require passing the price data
            # and looking up the closing price for each open position on the current_date.
            # For this modification, we'll use the entry price as a placeholder.
            # To make it more realistic, we'd need to load the price data here.

            # For demonstration purposes, let's assume we have access to the price data.
            # In a real scenario, you would load the data here based on the symbol and current_date.
            # Example (conceptual):
            # current_price = get_closing_price(pos['symbol'], current_date, data_dir)
            # if current_price is not None:
            #     total_portfolio_value += current_price * pos['size']
            # else:
            #     # Fallback: use entry price if current price not available
            #     total_portfolio_value += pos['entry_price'] * pos['size']

            # Simplified approach using entry price as proxy for current price
            total_portfolio_value += pos['entry_price'] * pos['size']


        # Record total portfolio value at the end of the day
        date_wise_accounts.append({
            'date': current_date,
            'total_portfolio_value': total_portfolio_value,
            'unallocated_capital': unallocated_capital # Keep unallocated capital as well
        })

    # Handle any remaining open positions at the very end of the simulation period
    if open_positions:
        # Need the last closing price for symbols still open.
        # This function doesn't have access to historical data frames directly.
        # This is a limitation of simulating solely based on the trade signal list.
        # For a real backtest, the last price of the data for each symbol
        # would be needed here. As a simplification, we'll assume the exit_price
        # in the signal (which for 'END' reason is the last price) is the closing price.

        for remaining_pos in open_positions:
             # Use the stored exit_price (which should be the last close for 'END' signals)
            last_price = remaining_pos['exit_price'] # Simplified

            pnl = (last_price - remaining_pos['entry_price']) * remaining_pos['size']
            current_capital += pnl
            unallocated_capital += (remaining_pos['entry_price'] * remaining_pos['size']) + pnl # Add back initial capital + PnL
            executed_trades.append({
                'symbol': remaining_pos['symbol'],
                'entry_date': remaining_pos['entry_date'],
                'entry_price': remaining_pos['entry_price'],
                'exit_date': remaining_pos['exit_date'], # This will be the original end date
                'exit_price': last_price,
                'size': remaining_pos['size'],
                'pnl': pnl,
                'reason': remaining_pos['reason'],
                'entry_type': remaining_pos['entry_type']
            })
        open_positions_count = 0 # All positions are now closed in simulation

    # Check for duplicate entries in the 'executed_trades' list
    seen = set()
    duplicates = []
    for trade in executed_trades:
        # Create a tuple of the trade's key attributes to check for duplicates
        trade_key = (trade['symbol'], trade['entry_date'], trade['entry_price'], trade['exit_date'], trade['exit_price'], trade['reason'], trade['entry_type'])
        if trade_key in seen:
            duplicates.append(trade)
        else:
            seen.add(trade_key)

    if duplicates:
        print("Duplicate executed trades found:")
        for dup in duplicates:
            print(dup)

    # Calculate largest loss streak and its dates
    current_loss_streak = 0
    largest_loss_streak = 0
    current_loss_streak_start_date = None
    largest_loss_streak_start_date = None
    largest_loss_streak_end_date = None

    # Sort executed_trades by exit date to correctly calculate streaks
    executed_trades.sort(key=lambda x: x['exit_date'])

    for trade in executed_trades:
        if trade['pnl'] <= 0:
            if current_loss_streak == 0: # Start of a new loss streak
                current_loss_streak_start_date = trade['exit_date']
            current_loss_streak += 1
        else:
            if current_loss_streak > largest_loss_streak:
                largest_loss_streak = current_loss_streak
                largest_loss_streak_start_date = current_loss_streak_start_date
                largest_loss_streak_end_date = trade['exit_date'] # The winning trade's exit date is the end of the loss streak
            current_loss_streak = 0
            current_loss_streak_start_date = None

    # Check if the largest loss streak is at the very end of the trades
    if current_loss_streak > largest_loss_streak:
        largest_loss_streak = current_loss_streak
        largest_loss_streak_start_date = current_loss_streak_start_date
        if executed_trades:
            largest_loss_streak_end_date = executed_trades[-1]['exit_date']

    # Ensure the last date's unallocated capital is recorded if it wasn't already by an exit
    # This part might need adjustment depending on whether the last date in all_dates
    # corresponds to an exit or just the end of the simulation period.
    # Let's ensure the last date in all_dates is captured if it's not already the exit date of the last trade.
    if all_dates:
        last_simulation_date = all_dates[-1]
        if not date_wise_accounts or date_wise_accounts[-1]['date'] < last_simulation_date:
             # Calculate total portfolio value for the absolute last date
             total_portfolio_value = unallocated_capital
             # Need last closing prices for remaining open positions
             # This requires loading data, which this function doesn't do directly.
             # Using entry price as a proxy again, or ideally, loading last prices.
             for pos in open_positions:
                 total_portfolio_value += pos['entry_price'] * pos['size'] # Simplified

             date_wise_accounts.append({
                'date': last_simulation_date,
                'total_portfolio_value': total_portfolio_value,
                'unallocated_capital': unallocated_capital
             })


    print("Max positions - ", max_positions)
    # Check if largest_loss_streak_start_date and largest_loss_streak_end_date are not None before accessing strftime
    if largest_loss_streak_start_date is not None and largest_loss_streak_end_date is not None:
        print(f"Largest loss streak - {largest_loss_streak} (from {largest_loss_streak_start_date.strftime('%Y-%m-%d')} to {largest_loss_streak_end_date.strftime('%Y-%m-%d')})")
    else:
        print(f"Largest loss streak - {largest_loss_streak} (No dates available)")


    return executed_trades, current_capital, open_positions_count, date_wise_accounts

In [12]:
executed_trades, final_capital, final_open_positions_count, date_wise_accounts = simulate_trades(unique_trade_signals, STARTING_CAPITAL)

# Create DataFrame from executed trades
trades_df = pd.DataFrame(executed_trades)

# Create DataFrame for date-wise accounts
accounts_df = pd.DataFrame(date_wise_accounts)
# Ensure 'date' is a datetime object and set as index
accounts_df['date'] = pd.to_datetime(accounts_df['date'])
accounts_df = accounts_df.set_index('date').sort_index()

# Calculate daily portfolio value changes (including unallocated capital)
# This is a simplified approach, a true portfolio value would include open positions' market value.
# For this backtest, we use unallocated capital as a proxy for strategy equity growth.
accounts_df['daily_return'] = accounts_df['total_portfolio_value'].pct_change().fillna(0)

# Calculate Sharpe Ratio (assuming risk-free rate is 0 for simplicity)
# Annualization factor based on the number of trading days in the backtest
trading_days = len(accounts_df)
# Estimate years in backtest to get average trading days per year
if not accounts_df.empty:
    time_span_years = (accounts_df.index[-1] - accounts_df.index[0]).days / 365.25
    annualization_factor = trading_days / time_span_years if time_span_years > 0 else 252 # Fallback to 252 if time span is zero
else:
    annualization_factor = 252 # Fallback to 252 if accounts_df is empty


if not accounts_df.empty and accounts_df['daily_return'].std() != 0 and annualization_factor > 0:
    sharpe_ratio = accounts_df['daily_return'].mean() / accounts_df['daily_return'].std() * math.sqrt(annualization_factor)
else:
    sharpe_ratio = 0.0


# Calculate summary statistics from executed trades
total_trades_overall = trades_df.shape[0] if not trades_df.empty else 0
wins_overall = trades_df[trades_df['pnl'] > 0].shape[0] if not trades_df.empty else 0
losses_overall = trades_df[trades_df['pnl'] <= 0].shape[0] if not trades_df.empty else 0
winrate_overall = (wins_overall / total_trades_overall * 100) if total_trades_overall else 0.0
total_pnl_overall = trades_df['pnl'].sum() if not trades_df.empty else 0.0


overall_summary = {
    'start_capital': STARTING_CAPITAL,
    'end_capital': final_capital,
    'total_pnl': total_pnl_overall,
    'trades': total_trades_overall,
    'wins': wins_overall,
    'losses': losses_overall,
    'winrate': winrate_overall,
    'sharpe_ratio': sharpe_ratio # Add Sharpe Ratio to summary
}
summary_df = pd.DataFrame([overall_summary])


# save outputs
trades_df.to_csv(OUT_TRADES_CSV, index=False)
summary_df.to_csv(OUT_SUMMARY_CSV, index=False)
accounts_df.to_csv("date_wise_accounts.csv", index=True) # Save with date index


print("Backtest complete.")
print("Saved:", OUT_TRADES_CSV, OUT_SUMMARY_CSV, "date_wise_accounts.csv")

# show overall backtest summary
if not summary_df.empty:
    print("\nOverall Backtest Summary:")
    display(summary_df)
else:
    print("No successful trades to summarize.")

Max positions -  74
Largest loss streak - 32 (from 2022-12-19 to 2023-03-16)
Backtest complete.
Saved: nifty_backtest_trades.csv nifty_backtest_summary.csv date_wise_accounts.csv

Overall Backtest Summary:


,start_capital,end_capital,total_pnl,trades,wins,losses,winrate,sharpe_ratio
0,2500000.0,4.884291e+06,2.384291e+06,469,240,229,51.172708,2.674953


## Summary:

### Data Analysis Key Findings

*   The project implements a quantitative trading strategy using DMA 20, DMA 50, ADX, ATR, and OBV indicators on Nifty 500 stocks.
*   The strategy includes specific entry conditions based on ADX > 25, DMA 20 > DMA 50, price touching DMA 20/50, OBV > OBV MA 20, and +DI > -DI, with separate rules for initial and pyramid entries.
*   Exit conditions are defined by a Stop Loss (DMA 50 - ATR * 1.5), DMA 20 crossing below DMA 50, or the end of the backtest period.
*   Risk management involves position sizing based on a percentage of maximum capital (2%), aiming for 0.5% risk per trade, applying a minimum 1.5 risk-reward ratio filter, and limiting open positions to 250.
*   The backtest simulates trades over a 3-year period ending in 2024, starting with 2,500,000 capital.
*   Key backtest results, including Total PnL, Winrate, Sharpe Ratio, and Largest Loss Streak, are saved to `nifty_backtest_summary.csv`. Detailed trade logs are in `nifty_backtest_trades.csv`, and daily account values in `date_wise_accounts.csv`.

### Insights or Next Steps

*   Analyze the `nifty_backtest_summary.csv` to evaluate the strategy's historical performance and identify areas for improvement, such as optimizing indicator parameters or refining entry/exit logic.
*   Consider adding transaction costs and slippage to the simulation to obtain a more realistic assessment of the strategy's profitability in live trading.
